<a href="https://colab.research.google.com/github/JairAlbertoHuertaDiaz45/Simulaci-n-II/blob/main/Ejercicio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np
from scipy.stats import norm

**Definimos los parámetros del problema**

In [17]:
# Precio inicial del activo
S0 = 150

# Precio de ejercicio
K = 150

# Tasa libre de riesgo anual (capitalización continua)
r = 0.08

# Volatilidad anual del activo
sigma = 0.35

# Tiempo al vencimiento (6 meses)
T = 0.5

**Definimos una función para valuar un Put Europeo con Black–Scholes**

In [18]:
def black_scholes_put(S, K, r, sigma, T):
    """
    Calcula el valor de una opción Put europea utilizando
    el modelo de Black y Scholes.
    """
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    put_price = K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
    return put_price


# Se obtiene el valor de referencia del Put europeo
P_BS = black_scholes_put(S0, K, r, sigma, T)

**Función para valuar un Put con el modelo binomial CRR**

In [19]:
def binomial_put(S0, K, r, sigma, T, N, americana=False):
    """
    Valuación de una opción Put mediante el modelo binomial
    de Cox-Ross-Rubinstein. Permite opción europea o americana.
    """
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u

    # Probabilidad neutral al riesgo
    p = (np.exp(r * dt) - d) / (u - d)

    # Precios del activo en el vencimiento
    ST = np.array([S0 * (u**j) * (d**(N-j)) for j in range(N+1)])

    # Payoff del Put en el vencimiento
    payoff = np.maximum(K - ST, 0)

    # Inducción hacia atrás
    for i in range(N-1, -1, -1):
        payoff = np.exp(-r*dt) * (p * payoff[1:] + (1-p) * payoff[:-1])

        # Evaluación de ejercicio anticipado (opción americana)
        if americana:
            ST = np.array([S0 * (u**j) * (d**(i-j)) for j in range(i+1)])
            payoff = np.maximum(payoff, K - ST)

    return payoff[0]

**Realizamos la comparación entre el modelo binomial y el modelo Black and Sholes**

Se busca el primer número de periodos para el cual, la diferencia absoluta sea al menos 0.0001

In [20]:
tolerancia = 0.0001
N_encontrado = None
diferencia = None

for N in range(1, 1000):
    P_binomial = binomial_put(S0, K, r, sigma, T, N)
    diff = abs(P_binomial - P_BS)

    if diff <= tolerancia:
        N_encontrado = N
        diferencia = diff
        break

***Resultados de comparación***

In [21]:
print("EJERCICIO 3 – RESULTADOS\n")
print(f"Put europeo (Black–Scholes): {P_BS:.6f}")

if N_encontrado is not None:
    print(f"Primer número de periodos N con diferencia ≤ 0.0001: {N_encontrado}")
    print(f"Diferencia encontrada: {diferencia:.6f}")
else:
    print("No se encontró convergencia con tolerancia 0.0001 en el rango analizado.")

EJERCICIO 3 – RESULTADOS

Put europeo (Black–Scholes): 11.729618
No se encontró convergencia con tolerancia 0.0001 en el rango analizado.


**Evaluación del PUT Americano**

In [22]:
N_americano = 70
P_put_americano = binomial_put(S0, K, r, sigma, T, N_americano, americana=True)

dt = T / N_americano
u = np.exp(sigma * np.sqrt(dt))
d = 1 / u

# Precio del activo en el nodo S0*u^20*d^50
S_nodo = S0 * (u**20) * (d**50)
valor_intrinseco = max(K - S_nodo, 0)

print("\nPut Americano:")
print(f"Precio del activo en el nodo S0*u^20*d^50: {S_nodo:.4f}")
print(f"Valor intrínseco en el nodo: {valor_intrinseco:.4f}")
print(f"Valor del Put Americano: {P_put_americano:.6f}")

if valor_intrinseco > P_put_americano:
    print("Conclusión: es óptimo ejercer anticipadamente en ese nodo.")
else:
    print("Conclusión: no es óptimo ejercer anticipadamente en ese nodo.")


Put Americano:
Precio del activo en el nodo S0*u^20*d^50: 61.7580
Valor intrínseco en el nodo: 88.2420
Valor del Put Americano: 12.300297
Conclusión: es óptimo ejercer anticipadamente en ese nodo.
